# Generate Database

### Drop and Create Database

In [ ]:
USE [master]
DROP DATABASE IF EXISTS DW2022
CREATE DATABASE DW2022
GO


### Drop and Create Schema

In [ ]:
USE [DW2022]

DROP SCHEMA IF EXISTS [DW]
GO

CREATE SCHEMA [DW]
GO

### Dimensions

In [ ]:
-- drop dimension dim_curso
DROP TABLE IF EXISTS [DW2022].[DW].[dim_curso]
GO

In [ ]:
-- create dimension dim_curso
IF NOT EXISTS (SELECT [name] FROM sys.tables WHERE [name] = 'dim_curso')
CREATE TABLE [DW2022].[DW].[dim_curso](
    [id]            INT IDENTITY(1,1) NOT NULL,
    [codigo]        VARCHAR(50) NOT NULL,
    [titulo]        VARCHAR(255) NOT NULL,
    [capacidad]     INT NOT NULL,
    [modalidad]     CHAR NOT NULL,
    [fechaInicio]   DATE NULL,
    [fechaFin]      DATE NULL,
    CONSTRAINT [PK_Curso_Id] PRIMARY KEY NONCLUSTERED (id)
)
GO

In [23]:
-- drop dimension dim_sala
DROP TABLE IF EXISTS [DW2022].[DW].[dim_sala]
GO

Commands completed successfully.

Total execution time: 00:00:00.010

In [24]:
-- create dimension dim_sala
IF NOT EXISTS (SELECT * FROM sys.tables WHERE [name] = 'dim_sala')
CREATE TABLE [DW2022].[DW].[dim_sala](
    [id]        INT IDENTITY(1,1) NOT NULL,
    [codigo]    VARCHAR(50) NOT NULL,
    [ubicacion] VARCHAR(50) NOT NULL,
    [nombre]    VARCHAR(255) NOT NULL,
    [tipo]      VARCHAR(10) NOT NULL,
    [asientos]  INT NOT NULL,
    [fecha_inicio] DATETIME NULL,
    [fecha_fin]    DATETIME NULL,
    CONSTRAINT [PK_Sala_Id] PRIMARY KEY NONCLUSTERED (id)
)
GO

Commands completed successfully.

Total execution time: 00:00:00.023

In [ ]:
-- drop dimension dim_profesor
DROP TABLE IF EXISTS [DW2022].[DW].[dim_profesor]
GO

In [ ]:
-- create dimension dim_profesor
IF NOT EXISTS (SELECT * FROM sys.tables WHERE [name] = 'dim_profesor')
CREATE TABLE [DW2022].[DW].[dim_profesor](
    [id]                    INT IDENTITY(1,1) NOT NULL,
    [codigo]                VARCHAR(50) NOT NULL,
    [nombre]                VARCHAR(255) NOT NULL,
    [fechaNacimiento]       INT NOT NULL,
    [cantidadGrupoFamiliar] INT NOT NULL,
    [cantidadHijos]         INT NOT NULL,
    [comision]              FLOAT NOT NULL,
    [razon]                 FLOAT NOT NULL,
    [zip]                   VARCHAR(50) NOT NULL,  
    [estado]                VARCHAR(255) NOT NULL,
    [ciudad]                VARCHAR(255) NOT NULL,
    [calle]                 VARCHAR(255) NOT NULL,
    [fechaInicio]           DATE NOT NULL,
    [fechaFin]              DATE NOT NULL
    CONSTRAINT [PK_Profesor_Id] PRIMARY KEY NONCLUSTERED (id)
)
GO

In [ ]:
-- drop dimension dim_tiempo
DROP TABLE IF EXISTS [DW2022].[DW].[dim_tiempo]
GO

In [ ]:
-- create dimension dim_tiempo
IF NOT EXISTS (SELECT * FROM sys.tables WHERE [name] = 'dim_tiempo')
CREATE TABLE [DW2022].[DW].[dim_tiempo](
    [id]    INT IDENTITY(1,1) NOT NULL,
    [fecha] DATETIME NOT NULL,
    [diaNombre] VARCHAR(20) NOT NULL,
    [anio]  SMALLINT NOT NULL,
    [temporada] VARCHAR(15) NOT NULL
    CONSTRAINT [PK_Tiempo_Id] PRIMARY KEY NONCLUSTERED (id)
)
GO

Fact Tables

In [ ]:
-- drop fact_inscripciones
DROP TABLE IF EXISTS [DW2022].[DW].[fact_inscripciones]
GO

In [ ]:
-- create fact_inscripciones
IF NOT EXISTS (SELECT * FROM sys.tables WHERE [name] = 'fact_inscripciones')
CREATE TABLE [DW2022].[DW].[fact_inscripciones](
    [cursoId]       INT NOT NULL,
    [salaId]        INT NOT NULL,
    [profesorId]    INT NOT NULL,
    [tiempoId]      INT NOT NULL,
    [inscriptosM]   INT NOT NULL,
    [inscriptosF]   INT NOT NULL,
    [inscriptosT]   INT NOT NULL,
    CONSTRAINT [Fk_Curso_Id] FOREIGN KEY ([cursoId]) REFERENCES [DW2022].[DW].[dim_curso](id),
    CONSTRAINT [Fk_Sala_Id] FOREIGN KEY ([salaId]) REFERENCES [DW2022].[DW].[dim_sala](id),
    CONSTRAINT [Fk_Profesor_Id] FOREIGN KEY ([profesorId]) REFERENCES [DW2022].[DW].[dim_profesor](id),
    CONSTRAINT [Fk_Tiempo_Id] FOREIGN KEY ([tiempoId]) REFERENCES [DW2022].[DW].[dim_tiempo](id),
)